In [1]:
import pandas as pd
import numpy as np
import json
import requests
import xmltodict
import time
import pickle
import os
import sqlalchemy
import urllib.request
from mutagen.mp3 import MP3

In [2]:
config = json.load(open('../config.json', 'r'))

In [3]:
user = config['db']['username']
password = config['db']['password']
host = config['db']['host']
port = config['db']['port']
dbname = config['db']['dbname']

In [4]:
template = "mysql+pymysql://{user}:{password}@{host}:{port}/{dbname}"
connection_string = template.format(user=user, password=password, host=host, port=port, dbname=dbname)

In [5]:
conn = sqlalchemy.create_engine(connection_string, pool_size=1)

In [29]:
if os.path.isfile(fname):
    os.remove(fname)

In [30]:
fname = 'feed.xml'
url = 'http://dataskeptic.com/feed.rss'
url = 'http://dataskeptic.libsyn.com/rss'

if not(os.path.isfile(fname)):
    print('fetching')
    r = requests.get(url)
    f = open(fname, 'wb')
    f.write(r.text.encode('utf-8'))
    f.close()

with open(fname) as fd:
    xml = xmltodict.parse(fd.read())

fetching


In [31]:
conn.execute("UPDATE blog set publish_Date=NoW() where blog_id=389")

In [32]:
emap = {}
episodes = xml['rss']['channel']['item']
for episode in episodes:
    guid = episode['guid']['#text']
    url = episode['enclosure']['@url']
    emap[guid] = url

In [40]:
q = """
SELECT t1.blog_id, prettyname, guid
 , t2.content_id as mp3_content, t3.content_id as homepage_content
 , t1.title, t1.abstract
FROM blog t1
LEFT JOIN related_content t2
 on t1.blog_id = t2.blog_id
 and t2.type='mp3'
LEFT JOIN related_content t3
 on t1.blog_id = t3.blog_id
 and t3.type='homepage-image'
WHERE t1.prettyname like '/episodes/%%'
AND (t2.blog_id is NULL or t3.blog_id is null)
LIMIT 10
"""
df = pd.read_sql(q, conn)

In [41]:
df.shape

(0, 7)

In [35]:
tpl = """
INSERT INTO related_content (blog_id, dest, type, title, body, duration) VALUES 
({blog_id}, '{dest}', '{type}', '{title}', '{body}', {duration})
"""

In [36]:
dest

NameError: name 'dest' is not defined

In [37]:
for r in range(df.shape[0]):
    row = df.iloc[r]
    blog_id = row['blog_id']
    guid = row['guid']
    title = row['title'].replace("'", "\\'")
    body = row['abstract'].replace("'", "\\'")
    dest = emap[guid]
    response = urllib.request.urlopen(dest)
    data = response.read()
    fname = 'temp.mp3'
    f = open(fname, 'wb')
    f.write(data)
    f.close()
    audio = MP3(fname)
    duration = int(audio.info.length)
    q1 = tpl.format(blog_id=blog_id, dest=dest, type='mp3', title=title, body=body, duration=duration)
    r1 = conn.execute(q1)
    dest = 'https://s3.amazonaws.com/dataskeptic.com/img/2018/ai-cover.png'
    q2 = tpl.format(blog_id=blog_id, dest=dest, type='homepage-image', title=title, body=body, duration=-1)
    r2 = conn.execute(q2)

In [38]:
#conn.execute("delete from related_content where blog_id=382")

In [39]:
pd.read_sql("SELECT * from related_content order by content_id desc limit 10", conn)

,content_id,blog_id,dest,type,title,body,created_date,blog_id2,duration
0,674,389,https://s3.amazonaws.com/dataskeptic.com/img/2...,homepage-image,Kalman Filters,A Kalman Filter is a technique for taking a se...,2018-06-01 14:02:58,NaN,-1.0
1,673,389,http://traffic.libsyn.com/dataskeptic/kalman-f...,mp3,Kalman Filters,A Kalman Filter is a technique for taking a se...,2018-06-01 14:02:58,NaN,1291.0
2,672,388,https://s3.amazonaws.com/dataskeptic.com/img/2...,homepage-image,ELO,,2018-05-28 19:00:07,NaN,NaN
3,671,387,https://s3.amazonaws.com/dataskeptic.com/img/2...,homepage-image,AI in Industry,This episode recaps some of the AI news from M...,2018-05-25 15:06:48,NaN,-1.0
4,670,387,http://traffic.libsyn.com/dataskeptic/ms-ai.mp...,mp3,AI in Industry,This episode recaps some of the AI news from M...,2018-05-25 15:06:47,NaN,2582.0
5,669,387,https://s3.amazonaws.com/dataskeptic.com/guest...,person,Carlos Pessoa,Carlos Pessoa is the Principal Software Engine...,2018-05-25 06:14:46,-1.0,NaN
6,668,387,https://s3.amazonaws.com/dataskeptic.com/guest...,person,Steve Guggenheimer,Steve Guggenheimer is the corporate vice presi...,2018-05-25 06:14:12,-1.0,NaN
7,667,386,https://s3.amazonaws.com/dataskeptic.com/img/2...,homepage-image,,,2018-05-21 15:46:16,-1.0,NaN
8,666,385,https://s3.amazonaws.com/dataskeptic.com/guest...,person,Julian Togelius,Julian Togelius is an associate professor of C...,2018-05-18 17:10:09,-1.0,NaN
9,665,385,https://s3.amazonaws.com/dataskeptic.com/guest...,person,Georgios Yannakakis,Georgios Yannakakis is a professor and directo...,2018-05-18 17:06:26,-1.0,NaN
